In [5]:
from google.colab import drive
drive.mount('/content/drive')

# import sys
# sys.path.append('/content/drive/MyDrive/Major_Project_Cse')

%cd "/content/drive/MyDrive/Major_Project_Cse"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Major_Project_Cse


In [6]:
import pandas as pd

# read the data from the CSV file
data = pd.read_csv('train_data.csv')

# load the keypoints and labels into separate variables
# train_keypoints = list(data['keypoints'])
train_keypoints=[]
train_labels = list(data['label'])

In [7]:
numOfKeypoints = 34
import ast
for keypoints_str in data['keypoints']:
    keypoints_list = ast.literal_eval(keypoints_str)
    train_keypoints.append(keypoints_list)

In [8]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import initializers
from keras.layers import Conv3D, MaxPooling3D,Dense,ZeroPadding3D,Flatten,Activation,BatchNormalization,RandomFlip,RandomRotation,Rescaling
from keras.models import load_model


In [9]:
type(train_keypoints[0][0][1][0])

NoneType

In [10]:
# calculate the center of the object in each frame
centers = []
for keypoints in train_keypoints:
    frame_centers = []
    for frame_keypoints in keypoints:
        # assume the center is the mean of all keypoints
       center_x = np.nanmean([kp[0] for kp in frame_keypoints if kp[0] is not None])
       center_y = np.nanmean([kp[1] for kp in frame_keypoints if kp[1] is not None])
       frame_centers.append((center_x, center_y))
    centers.append(frame_centers)

train_relative_keypoints = train_keypoints
# make the keypoints relative to the center
for i in range(len(train_relative_keypoints)):
    for j in range(len(train_relative_keypoints[i])):
        for k in range(len(train_relative_keypoints[i][j])):
            if train_relative_keypoints[i][j][k][0] is not None:
               train_relative_keypoints[i][j][k][0] -= centers[i][j][0]
            if train_relative_keypoints[i][j][k][1] is not None:
              train_relative_keypoints[i][j][k][1] -= centers[i][j][1]


<ipython-input-10-a6c48355f099>:7: RuntimeWarning: Mean of empty slice
  center_x = np.nanmean([kp[0] for kp in frame_keypoints if kp[0] is not None])
<ipython-input-10-a6c48355f099>:8: RuntimeWarning: Mean of empty slice
  center_y = np.nanmean([kp[1] for kp in frame_keypoints if kp[1] is not None])


In [11]:
count=0
new_train_keypoints = []
new_train_labels = []
for i in range(len(train_relative_keypoints)):
    flag=1
    frames_keypoints=[]
    for j in range(len(train_relative_keypoints[i])):
        for k in range(len(train_relative_keypoints[i][j])):
            if train_relative_keypoints[i][j][k][0] is None or train_relative_keypoints[i][j][k][1] is None:
              count+=1
              flag=0
              break
        if(flag==1):
          frames_keypoints.append(train_relative_keypoints[i][j])
    if(len(frames_keypoints)>0):
      kk = [[0.0,0.0] for _ in range(numOfKeypoints)]
      for i in range (16-len(frames_keypoints)):
          frames_keypoints.append(kk)
      # if(len(frames_keypoints)!=16):
      #   print("2 : ",len(frames_keypoints))
      new_train_keypoints.append(frames_keypoints)
      new_train_labels.append(train_labels[i])

In [12]:
print(len(new_train_keypoints),len(new_train_labels))

984 984


In [13]:
# train_videos = np.asarray(train_videos)
new_train_keypoints = np.asarray(new_train_keypoints)
new_train_labels = np.asarray(new_train_labels).astype('int32')

# train_keypoints = tf.convert_to_tensor(train_keypoints, dtype=tf.float32)
# # yKey_train = tf.cast(yKey_train , dtype=tf.float32)
# train_labels = tf.convert_to_tensor(train_labels, dtype=tf.float32)

# train_keypoints = np.array([np.array(val) for val in train_keypoints])

from sklearn.model_selection import train_test_split

XKey_train,XKey_test,yKey_train,yKey_test = train_test_split(new_train_keypoints,new_train_labels,test_size=0.25,shuffle=True)

In [14]:
print(len(XKey_train),len(XKey_test))

738 246


In [15]:
XKey_train = tf.convert_to_tensor(XKey_train, dtype=tf.float32)
yKey_train = tf.convert_to_tensor(yKey_train, dtype=tf.int32)

In [81]:
from keras import models, layers
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, TimeDistributed, GlobalAveragePooling1D
from keras.optimizers import Adam
from keras.initializers import GlorotNormal
from keras.regularizers import L1L2

yKey_train = np.squeeze(yKey_train)

tf.config.run_functions_eagerly(True)

# Define model input shape
input_shape = (16, 34, 2)

# Build model architecture
model = models.Sequential()

model.add(TimeDistributed(Conv1D(filters=16, kernel_size=3, activation='relu'),input_shape = input_shape))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=128, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=50, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))

# flatten output from CNN layers
model.add(TimeDistributed(GlobalAveragePooling1D()))

# add LSTM layer
model.add(LSTM(100, dropout=0.0, recurrent_dropout=0.0, kernel_initializer=GlorotNormal(seed=42), recurrent_regularizer=L1L2(l1=0.0, l2=0.001), bias_regularizer=L1L2(l1=0.0, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))
# model.add(LSTM(100, recurrent_regularizer=L1L2(l1=0.01, l2=0.001), bias_regularizer=L1L2(l1=0.01, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))

# TODO: forget bias 
# TODO: L1 = 0.01


# add fully connected layers

# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(80, activation='relu'))

# add Softmax layer
model.add(Dense(20, activation='softmax'))

# compile model
adam = Adam(learning_rate=0.0001) 
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# print model summary
# print(model.summary())

model.fit(XKey_train,yKey_train,batch_size=32,epochs = 300,validation_split = 0.2)

Epoch 1/300
19/19 [==============================] - 5s 273ms/step - loss: 3.2523 - accuracy: 0.0559 - val_loss: 3.2460 - val_accuracy: 0.1622
Epoch 2/300
19/19 [==============================] - 4s 203ms/step - loss: 3.2425 - accuracy: 0.1102 - val_loss: 3.2360 - val_accuracy: 0.1824
Epoch 3/300
19/19 [==============================] - 4s 193ms/step - loss: 3.2327 - accuracy: 0.1356 - val_loss: 3.2260 - val_accuracy: 0.1757
Epoch 4/300
19/19 [==============================] - 5s 243ms/step - loss: 3.2231 - accuracy: 0.1356 - val_loss: 3.2148 - val_accuracy: 0.1824
Epoch 5/300
19/19 [==============================] - 4s 222ms/step - loss: 3.2115 - accuracy: 0.1390 - val_loss: 3.2029 - val_accuracy: 0.1959
Epoch 6/300
19/19 [==============================] - 4s 203ms/step - loss: 3.1981 - accuracy: 0.1407 - val_loss: 3.1855 - val_accuracy: 0.1824
Epoch 7/300
19/19 [==============================] - 4s 197ms/step - loss: 3.1803 - accuracy: 0.1051 - val_loss: 3.1572 - val_accuracy: 0.1149

In [ ]:
# import mediapipe as mp

# landmarks = mp_pose.PoseLandmark.__members__
# print(landmarks)


{'NOSE': <PoseLandmark.NOSE: 0>, 'LEFT_EYE_INNER': <PoseLandmark.LEFT_EYE_INNER: 1>, 'LEFT_EYE': <PoseLandmark.LEFT_EYE: 2>, 'LEFT_EYE_OUTER': <PoseLandmark.LEFT_EYE_OUTER: 3>, 'RIGHT_EYE_INNER': <PoseLandmark.RIGHT_EYE_INNER: 4>, 'RIGHT_EYE': <PoseLandmark.RIGHT_EYE: 5>, 'RIGHT_EYE_OUTER': <PoseLandmark.RIGHT_EYE_OUTER: 6>, 'LEFT_EAR': <PoseLandmark.LEFT_EAR: 7>, 'RIGHT_EAR': <PoseLandmark.RIGHT_EAR: 8>, 'MOUTH_LEFT': <PoseLandmark.MOUTH_LEFT: 9>, 'MOUTH_RIGHT': <PoseLandmark.MOUTH_RIGHT: 10>, 'LEFT_SHOULDER': <PoseLandmark.LEFT_SHOULDER: 11>, 'RIGHT_SHOULDER': <PoseLandmark.RIGHT_SHOULDER: 12>, 'LEFT_ELBOW': <PoseLandmark.LEFT_ELBOW: 13>, 'RIGHT_ELBOW': <PoseLandmark.RIGHT_ELBOW: 14>, 'LEFT_WRIST': <PoseLandmark.LEFT_WRIST: 15>, 'RIGHT_WRIST': <PoseLandmark.RIGHT_WRIST: 16>, 'LEFT_PINKY': <PoseLandmark.LEFT_PINKY: 17>, 'RIGHT_PINKY': <PoseLandmark.RIGHT_PINKY: 18>, 'LEFT_INDEX': <PoseLandmark.LEFT_INDEX: 19>, 'RIGHT_INDEX': <PoseLandmark.RIGHT_INDEX: 20>, 'LEFT_THUMB': <PoseLandmark.

In [82]:
from keras import models, layers
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, TimeDistributed, GlobalAveragePooling1D
from keras.optimizers import Adam
from keras.initializers import GlorotNormal
from keras.regularizers import L1L2

yKey_train = np.squeeze(yKey_train)

tf.config.run_functions_eagerly(True)

# Define model input shape
input_shape = (16, 34, 2)

# Build model architecture
model = models.Sequential()

model.add(TimeDistributed(Conv1D(filters=16, kernel_size=3, activation='relu'),input_shape = input_shape))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=128, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=50, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))

# flatten output from CNN layers
model.add(TimeDistributed(GlobalAveragePooling1D()))

# add LSTM layer
# model.add(LSTM(100, dropout=0.0, recurrent_dropout=0.0, kernel_initializer=GlorotNormal(seed=42), recurrent_regularizer=L1L2(l1=0.0, l2=0.001), bias_regularizer=L1L2(l1=0.0, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))
model.add(LSTM(100, recurrent_regularizer=L1L2(l1=0.01, l2=0.001), bias_regularizer=L1L2(l1=0.01, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))

# TODO: forget bias 
# TODO: L1 = 0.01


# add fully connected layers

# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(80, activation='relu'))

# add Softmax layer
model.add(Dense(20, activation='softmax'))

# compile model
adam = Adam(learning_rate=0.001) 
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# print model summary
# print(model.summary())

model.fit(XKey_train,yKey_train,batch_size=32,epochs = 300,validation_split = 0.2)

Epoch 1/300
19/19 [==============================] - 4s 217ms/step - loss: 21.8459 - accuracy: 0.0593 - val_loss: 18.2877 - val_accuracy: 0.1284
Epoch 2/300
19/19 [==============================] - 4s 236ms/step - loss: 15.7570 - accuracy: 0.0864 - val_loss: 12.9646 - val_accuracy: 0.1757
Epoch 3/300
19/19 [==============================] - 5s 255ms/step - loss: 11.0284 - accuracy: 0.1390 - val_loss: 8.7999 - val_accuracy: 0.1622
Epoch 4/300
19/19 [==============================] - 4s 209ms/step - loss: 7.4553 - accuracy: 0.2356 - val_loss: 5.9011 - val_accuracy: 0.3243
Epoch 5/300
19/19 [==============================] - 4s 195ms/step - loss: 5.1357 - accuracy: 0.3203 - val_loss: 4.1856 - val_accuracy: 0.4054
Epoch 6/300
19/19 [==============================] - 5s 285ms/step - loss: 3.7443 - accuracy: 0.3644 - val_loss: 3.2042 - val_accuracy: 0.4730
Epoch 7/300
19/19 [==============================] - 4s 191ms/step - loss: 3.0305 - accuracy: 0.3831 - val_loss: 2.7632 - val_accuracy: 0

In [83]:
from keras import models, layers
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, TimeDistributed, GlobalAveragePooling1D
from keras.optimizers import Adam
from keras.initializers import GlorotNormal
from keras.regularizers import L1L2

yKey_train = np.squeeze(yKey_train)

tf.config.run_functions_eagerly(True)

# Define model input shape
input_shape = (16, 34, 2)

# Build model architecture
model = models.Sequential()

model.add(TimeDistributed(Conv1D(filters=16, kernel_size=3, activation='relu'),input_shape = input_shape))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=128, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=50, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))

# flatten output from CNN layers
model.add(TimeDistributed(GlobalAveragePooling1D()))

# add LSTM layer
model.add(LSTM(100, dropout=0.0, recurrent_dropout=0.0, kernel_initializer=GlorotNormal(seed=42), recurrent_regularizer=L1L2(l1=0.0, l2=0.001), bias_regularizer=L1L2(l1=0.0, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))
# model.add(LSTM(100, recurrent_regularizer=L1L2(l1=0.01, l2=0.001), bias_regularizer=L1L2(l1=0.01, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))

# TODO: forget bias 
# TODO: L1 = 0.01


# add fully connected layers

# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(80, activation='relu'))

# add Softmax layer
model.add(Dense(20, activation='softmax'))

# compile model
adam = Adam(learning_rate=0.0001) 
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# print model summary
# print(model.summary())

model.fit(XKey_train,yKey_train,batch_size=32,epochs = 500,validation_split = 0.2)

Epoch 1/500
19/19 [==============================] - 8s 428ms/step - loss: 3.2529 - accuracy: 0.1051 - val_loss: 3.2474 - val_accuracy: 0.1554
Epoch 2/500
19/19 [==============================] - 10s 502ms/step - loss: 3.2432 - accuracy: 0.1237 - val_loss: 3.2367 - val_accuracy: 0.1486
Epoch 3/500
19/19 [==============================] - 6s 316ms/step - loss: 3.2331 - accuracy: 0.1475 - val_loss: 3.2263 - val_accuracy: 0.1959
Epoch 4/500
19/19 [==============================] - 5s 275ms/step - loss: 3.2234 - accuracy: 0.1492 - val_loss: 3.2156 - val_accuracy: 0.1757
Epoch 5/500
19/19 [==============================] - 4s 205ms/step - loss: 3.2123 - accuracy: 0.1492 - val_loss: 3.2026 - val_accuracy: 0.1419
Epoch 6/500
19/19 [==============================] - 4s 206ms/step - loss: 3.1988 - accuracy: 0.1661 - val_loss: 3.1848 - val_accuracy: 0.2095
Epoch 7/500
19/19 [==============================] - 5s 268ms/step - loss: 3.1795 - accuracy: 0.1593 - val_loss: 3.1553 - val_accuracy: 0.195

In [16]:
from keras import models, layers
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, TimeDistributed, GlobalAveragePooling1D
from keras.optimizers import Adam
from keras.initializers import GlorotNormal
from keras.regularizers import L1L2

yKey_train = np.squeeze(yKey_train)

tf.config.run_functions_eagerly(True)

# Define model input shape
input_shape = (16, 34, 2)

# Build model architecture
model = models.Sequential()

model.add(TimeDistributed(Conv1D(filters=16, kernel_size=3, activation='relu'),input_shape = input_shape))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=128, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=50, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))

# flatten output from CNN layers
model.add(TimeDistributed(GlobalAveragePooling1D()))

# add LSTM layer
# model.add(LSTM(100, dropout=0.0, recurrent_dropout=0.0, kernel_initializer=GlorotNormal(seed=42), recurrent_regularizer=L1L2(l1=0.0, l2=0.001), bias_regularizer=L1L2(l1=0.0, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))
model.add(LSTM(100, recurrent_regularizer=L1L2(l1=0.01, l2=0.001), bias_regularizer=L1L2(l1=0.01, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))

# TODO: forget bias 
# TODO: L1 = 0.01


# add fully connected layers

# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(80, activation='relu'))

# add Softmax layer
model.add(Dense(20, activation='softmax'))

# compile model
adam = Adam(learning_rate=0.001) 
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# print model summary
# print(model.summary())

history = model.fit(XKey_train,yKey_train,batch_size=32,epochs = 500,validation_split = 0.2)

/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/500
19/19 [==============================] - 9s 410ms/step - loss: 21.8496 - accuracy: 0.0864 - val_loss: 18.3085 - val_accuracy: 0.1554
Epoch 2/500
19/19 [==============================] - 4s 193ms/step - loss: 15.7795 - accuracy: 0.1712 - val_loss: 13.0278 - val_accuracy: 0.1689
Epoch 3/500
19/19 [==============================] - 5s 264ms/step - loss: 11.0763 - accuracy: 0.2068 - val_loss: 8.9120 - val_accuracy: 0.2635
Epoch 4/500
19/19 [==============================] - 4s 212ms/step - loss: 7.5253 - accuracy: 0.2831 - val_loss: 6.0237 - val_accuracy: 0.3378
Epoch 5/500
19/19 [==============================] - 4s 208ms/step - loss: 5.1805 - accuracy: 0.3441 - val_loss: 4.2347 - val_accuracy: 0.3851
Epoch 6/500
19/19 [==============================] - 4s 234ms/step - loss: 3.8669 - accuracy: 0.3271 - val_loss: 3.2942 - val_accuracy: 0.4459
Epoch 7/500
19/19 [==============================] - 5s 257ms/step - loss: 3.0771 - accuracy: 0.3966 - val_loss: 2.7791 - val_accuracy: 0

KeyboardInterrupt: ignored

In [18]:


# Get model summary
model.summary()

# # Get training and validation accuracy
# train_accuracy = history.history['accuracy']
# val_accuracy = history.history['val_accuracy']

# # Get training and validation loss
# train_loss = history.history['loss']
# val_loss = history.history['val_loss']


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 16, 32, 16)       112       
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 16, 16, 16)       0         
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 16, 14, 128)      6272      
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 16, 7, 128)       0         
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 16, 5, 50)        1

NameError: ignored